In [3]:
import requests
from bs4 import BeautifulSoup
import re

def crawl_scu_teachers():
    # 1. 第一个网页（列表页）的URL
    list_url = 'https://cs.scu.edu.cn/szdw/rgznx.htm'
    resp = requests.get(list_url)
    resp.encoding = 'utf-8'  # 防止中文乱码
    soup = BeautifulSoup(resp.text, 'html.parser')

    # 2. 找到所有人的 <li>，提取姓名与详情链接
    #    你需要根据实际HTML结构，找到存放教师信息的那一段 <ul>/<li>。
    #    以下示例假设每个教师的 <li> 中有 <a> 标签。
    li_tags = soup.find_all('li')
    
    # 为了避免误抓，可以先过滤出真正包含“教师个人主页链接”的 li
    # 你也可以用更加精确的查找方式，比如 soup.select('ul.xxx > li') 等
    teacher_links = []
    for li in li_tags:
        a_tag = li.find('a')
        if a_tag and a_tag.get('href', '').endswith('.htm'):
            # 假设 <a> 标签文本就是人名
            name = a_tag.get_text(strip=True)
            relative_url = a_tag['href']
            # 拼接详情页URL
            # 如果链接是 "../info/1288/18682.htm" 这种形式，需要手动拼上域名
            full_url = 'https://cs.scu.edu.cn' + relative_url.replace('..', '')
            teacher_links.append((name, full_url))
    
    # 3. 依次访问每个教师详情页，获取浏览量
    for name, person_url in teacher_links:
        views = get_view_count(person_url)
        print(f'{name}：{views}')

def get_view_count(url):
    """
    给定某位老师详情页 URL，返回浏览量（若未找到则返回 '未知'）。
    """
    try:
        resp = requests.get(url)
        resp.encoding = 'utf-8'
        soup = BeautifulSoup(resp.text, 'html.parser')
        
        # 方式一：根据 script 标签的文本中包含 _showDynClicks("wbnews" 来找
        script_tag = soup.find('script', text=re.compile(r'_showDynClicks\("wbnews",'))
        if not script_tag:
            return '未知'
        
        # 方式二：找到 script 标签后，再找它的下一个兄弟标签 <span>（通常就是浏览次数）
        span_tag = script_tag.find_next_sibling('span')
        if not span_tag:
            return '未知'
        
        # 获取 span 文本，并去掉可能的 “次” 字
        view_text = span_tag.get_text(strip=True)
        # 如果末尾有“次”，可去掉
        view_text = view_text.replace('次', '').strip()
        
        return view_text
    except Exception as e:
        print(f'获取 {url} 时出错: {e}')
        return '未知'


get_view_count('https://cs.scu.edu.cn/szdw/rgznx/202012/t20201207_619515.htm')

C:\Users\shann\AppData\Local\Temp\ipykernel_10088\1872305724.py:46: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  script_tag = soup.find('script', text=re.compile(r'_showDynClicks\("wbnews",'))


'未知'